In [24]:
from PIL import Image 
from IPython.display import display 
import random
import json

In [25]:
# Variables

TOTAL_IMAGES = 250

all_images = []

background = ["blue", "gold", "green", "peach", "pink"]
color = ["black", "blue", "deepblue", "green",
         "orange", "purple", "red", "white", "yellow"]
spots = ["black", "blue", "white"]
floor = ["grass", "mud", "sand", "snow"]
toxicity = ["healthy", "toxic"]

# add magic mushroom, number of mushrooms, MAYBE add weather/landscape instead of floor?

background_weights = [10, 5, 15, 40, 30]
color_weights = [1, 2, 5, 15, 25, 11, 32, 1, 8]
spots_weights = [20, 5, 75]
floor_weights = [50, 30, 15, 5]
toxicity_weights = [85, 15]


In [26]:
def create_new_image():
    new_image = {}

    # For each trait category, select a random trait based on the weightings
    new_image["Background"] = random.choices(background, background_weights)[0]
    new_image["Color"] = random.choices(color, color_weights)[0]
    new_image["Spots"] = random.choices(spots, spots_weights)[0]
    new_image["Floor"] = random.choices(floor, floor_weights)[0]
    new_image["Toxicity"] = random.choices(toxicity, toxicity_weights)[0]

    if new_image in all_images:
        return create_new_image()
    else:
        return new_image


for i in range(TOTAL_IMAGES):
    new_image = create_new_image()

    all_images.append(new_image)


ValueError: The number of weights does not match the population

In [ ]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [ ]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [ ]:
# print(all_images)

In [ ]:
# Get Trait Counts

background_count = {}
for item in background:
    background_count[item] = 0

color_count = {}
for item in color:
    color_count[item] = 0

spots_count = {}
for item in spots:
    spots_count[item] = 0

floor_count = {}
for item in floor:
    floor_count[item] = 0

toxicity_count = {}
for item in toxicity:
    toxicity_count[item] = 0

for image in all_images:
    background_count[image["Background"]] += 1
    color_count[image["Color"]] += 1
    spots_count[image["Spots"]] += 1
    floor_count[image["Floor"]] += 1
    toxicity_count[image["Toxicity"]] += 1

print(background_count)
print(color_count)
print(spots_count)
print(floor_count)


{'blue': 49, 'gold': 36, 'green': 49, 'peach': 58, 'pink': 58}
{'black': 14, 'green': 33, 'orange': 53, 'purple': 49, 'red': 52, 'white': 11, 'yellow': 38}
{'black': 86, 'blue': 55, 'white': 109}
{'grass': 79, 'mud': 72, 'sand': 62, 'snow': 37}


In [ ]:
# Generate Metadata for all Traits
METADATA_FILE_NAME = './metadata/all-traits.json'
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)

print("Created all-traits JSON file")


Created all-traits JSON file


In [ ]:
# Generate Images
for item in all_images:

    # print("sourcing images for", item)

    # source images in their required order
    backgroundimage = Image.open(
        f'./trait-layers/backgrounds/{item["Background"]}.png').convert('RGBA')
    colorimage = Image.open(
        f'./trait-layers/colors/{item["Color"]}.png').convert('RGBA')
    shadingimage = Image.open(f'./trait-layers/shading.png').convert('RGBA')
    spotsimage = Image.open(
        f'./trait-layers/spots/{item["Spots"]}.png').convert('RGBA')
    outlineimage = Image.open(f'./trait-layers/outline.png').convert('RGBA')
    floorimage = Image.open(
        f'./trait-layers/floors/{item["Floor"]}.png').convert('RGBA')

    # change the alpha channel for shading / transparent images
    if item["Toxicity"] == "toxic":
        shadingimage.putalpha(128)

    # Create each composite
    com1 = Image.alpha_composite(backgroundimage, colorimage)
    com2 = Image.alpha_composite(com1, shadingimage)
    com3 = Image.alpha_composite(com2, spotsimage)
    com4 = Image.alpha_composite(com3, outlineimage)
    com5 = Image.alpha_composite(com4, floorimage)

    # Convert to RGB
    rgb_im = com5.convert('RGB')
    file_name = str(item["tokenId"]) + ".png"
    rgb_im.save("./images/" + file_name)


In [ ]:
#### Generate Metadata for each Image    

f = open('./metadata/all-traits.json',) 
data = json.load(f)


IMAGES_BASE_URI = "ADD_IMAGES_BASE_URI_HERE"
PROJECT_NAME = "ADD_PROJECT_NAME_HERE"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    token_id = i['tokenId']
    token = {
        "image": IMAGES_BASE_URI + str(token_id) + '.png',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' ' + str(token_id),
        "attributes": []
    }
    token["attributes"].append(getAttribute("Background", i["Background"]))
    token["attributes"].append(getAttribute("Color", i["Color"]))
    token["attributes"].append(getAttribute("Spots", i["Spots"]))
    token["attributes"].append(getAttribute("Floor", i["Floor"]))

    with open('./metadata/' + str(token_id), 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()